다음 데이터 세트를 이용하여 다이아몬드 가격예측(회귀) 

url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv" diamonds = pd.read_csv(url) 

1. ~ML (RF, DT, LR) 수행~

2. ~Dense layer만 이용 FNCC 구현~

3. 순환 데이터 변환후 CNN 구현



각 단계별로 수행완료후

검사받은 이후에 다음 단계 진행

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
url = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/diamonds.csv"
df = pd.read_csv(url)
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [3]:
print(np.unique(df['cut'], return_counts=True))
print(np.unique(df['color'], return_counts=True))
print(np.unique(df['clarity'], return_counts=True))

(array(['Fair', 'Good', 'Ideal', 'Premium', 'Very Good'], dtype=object), array([ 1610,  4906, 21551, 13791, 12082]))
(array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype=object), array([ 6775,  9797,  9542, 11292,  8304,  5422,  2808]))
(array(['I1', 'IF', 'SI1', 'SI2', 'VS1', 'VS2', 'VVS1', 'VVS2'],
      dtype=object), array([  741,  1790, 13065,  9194,  8171, 12258,  3655,  5066]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

origin_label_cut = df['cut'].copy()
origin_label_color = df['color'].copy()
origin_label_clarity = df['clarity'].copy()

df['cut'] = label_encoder.fit_transform(df['cut'])
df['color'] = label_encoder.fit_transform(df['color'])
df['clarity'] = label_encoder.fit_transform(df['clarity'])

In [5]:
df.shape

(53940, 10)

In [6]:
# 순환데이터 만들 때 대비해서 price를 제일 끝 column으로 이동
temp_price = df['price']
df = df.drop('price', axis=1)
df.shape

(53940, 9)

In [7]:
df.insert(9, 'price', temp_price)
df.shape

(53940, 10)

In [8]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.23,2,1,3,61.5,55.0,3.95,3.98,2.43,326
1,0.21,3,1,2,59.8,61.0,3.89,3.84,2.31,326
2,0.23,1,1,4,56.9,65.0,4.05,4.07,2.31,327
3,0.29,3,5,5,62.4,58.0,4.20,4.23,2.63,334
4,0.31,1,6,3,63.3,58.0,4.34,4.35,2.75,335
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,2,0,2,60.8,57.0,5.75,5.76,3.50,2757
53936,0.72,1,0,2,63.1,55.0,5.69,5.75,3.61,2757
53937,0.70,4,0,2,62.8,60.0,5.66,5.68,3.56,2757
53938,0.86,3,4,3,61.0,58.0,6.15,6.12,3.74,2757


In [9]:
len(df)

53940

# 1. 순환데이터 변환 후 CNN

In [10]:
import tensorflow as tf
from tensorflow import keras

2024-10-24 09:51:57.744346: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
X = df.drop('price', axis=1).to_numpy()
y = df['price'].to_numpy()

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    shuffle=True,
                                                    random_state=42)

print(f"X_train.shape: {X_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"y_test.shape: {y_test.shape}")

X_train.shape: (40455, 9)
X_test.shape: (13485, 9)
y_train.shape: (40455,)
y_test.shape: (13485,)


In [13]:
# scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
X_train_scaled.shape

(40455, 9)

In [15]:
# X와 y를 하나로 합친 dataset 만들기
# 합치기 위해서는 y도 2dim으로 변경해줘야됨 by reshape()
train_set = np.c_[X_train_scaled, y_train.reshape(-1, 1)]
test_set = np.c_[X_test_scaled, y_test.reshape(-1, 1)]

print(f"train_set.head(): \n{train_set[:5]}")

train_set.head(): 
[[-1.15666465e+00 -1.51349781e+00 -9.37210895e-01  1.83961348e+00
   2.20783668e+00  2.42414027e-01 -1.58998506e+00 -1.54444639e+00
  -1.36581585e+00  4.76000000e+02]
 [ 8.69167168e-02 -5.40643146e-01  2.00397806e+00 -1.06258138e+00
   3.85169052e-02 -6.54922789e-01  2.73560060e-01  2.91505677e-01
   2.82149480e-01  2.49300000e+03]
 [ 5.29547373e-01  4.32211514e-01  2.00397806e+00  6.78735533e-01
  -4.51329496e-01  2.42414027e-01  7.37217220e-01  6.76181348e-01
   6.34278825e-01  4.14500000e+03]
 [ 4.66314422e-01 -5.40643146e-01 -3.48973105e-01 -4.82142410e-01
  -7.31241725e-01 -6.54922789e-01  7.10467768e-01  6.67438719e-01
   5.77938130e-01  4.54100000e+03]
 [-3.97869240e-01 -5.40643146e-01 -3.48973105e-01  9.82965614e-02
   3.85169052e-02 -2.06254381e-01 -2.70345454e-01 -2.33052056e-01
  -2.39001950e-01  2.28300000e+03]]


In [16]:
train_set[1][9]

2493.0

## 순환데이터 만들기

In [17]:
# 2차원 - sequence에 속하는 샘플 중 마지막 label 값을 sequence의 label로 설정
def split_sequences(sequences, n_steps):
    # parameters
      # sequences: 기존 data instance들이 있는 array
      # n_steps  : 기존 data instance 몇 개를 하나로 묶을 것인지
    X = [] # sequential로 바뀔 array
    y = [] # sequential로 바뀔 array

    for i in range(len(sequences)): # 기존 array의 data instance 수
        end_idx = i + n_steps # (i+1)번 째 묶었을 때의 끝 index
        if (end_idx > len(sequences) - 1): break # 끝까지 다 묶으면 break

        # i부터 end_idx까지 즉, 묶을 data instance 단위를 말함
        # [[x1, x2, x3, y], [x1, x2, x3, y], [x1, x2, x3, y]]
        seq_x = sequences[i:end_idx, :-1]
        seq_y = sequences[end_idx-1, -1]

        # 묶은 애들을 Sequential Dataset의 instance로 추가
        X.append(seq_x)
        y.append(seq_y)

    return np.array(X), np.array(y) # 완성된 Sequential Dataset을 return

In [18]:
n_steps = 3

X_train_seq, y_train_seq = split_sequences(train_set, n_steps)
X_test_seq, y_test_seq = split_sequences(test_set, n_steps)

print(f"X_train_seq.shape: {X_train_seq.shape}", end=" ")
print(f"y_train_seq.shape: {y_train_seq.shape}")
print(f"X_test_seq.shape: {X_test_seq.shape}", end=" ")
print(f"y_test_seq.shape: {y_test_seq.shape}")

X_train_seq.shape: (40452, 3, 9) y_train_seq.shape: (40452,)
X_test_seq.shape: (13482, 3, 9) y_test_seq.shape: (13482,)


In [19]:
n_steps = X_train_seq.shape[1]
features = X_train_seq.shape[2]

In [24]:
y_train_seq[:5]

array([4145., 4541., 2283., 7243.,  924.])

## CNN

In [20]:
# Create the model
model = keras.models.Sequential()

# Add the layers to model
model.add(keras.layers.Conv1D(filters=64,
                              kernel_size=3,
                              activation='relu',
                              input_shape=(n_steps, features),
                              name='conv1d1'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Flatten())
# model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(units=128,
                             activation='relu',
                             name='dense1'))
# model.add(keras.layers.Dropout(0.3))
# model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(units=1,
                             name='output'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d1 (Conv1D)            (None, 1, 64)             1792      
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense1 (Dense)              (None, 128)               8320      
                                                                 
 output (Dense)              (None, 1)                 129       
                                                                 
Total params: 10,241
Trainable params: 10,241
Non-trainable params: 0
_________________________________________________________________


2024-10-24 09:51:59.968395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
from keras.optimizers import Adam

model.compile(Adam(learning_rate=0.004), 
              loss='mean_squared_error', 
              metrics=['mean_squared_error'])

history = model.fit(X_train_seq, y_train_seq,
                    validation_split=0.1,
                    batch_size=512,
                    shuffle=True,
                    verbose=2,
                    epochs=100)

Epoch 1/100
72/72 - 1s - loss: 28577020.0000 - mean_squared_error: 28577020.0000 - val_loss: 19541110.0000 - val_mean_squared_error: 19541110.0000 - 701ms/epoch - 10ms/step
Epoch 2/100
72/72 - 0s - loss: 7585775.0000 - mean_squared_error: 7585775.0000 - val_loss: 2946929.0000 - val_mean_squared_error: 2946929.0000 - 130ms/epoch - 2ms/step
Epoch 3/100
72/72 - 0s - loss: 2849982.7500 - mean_squared_error: 2849982.7500 - val_loss: 1786156.0000 - val_mean_squared_error: 1786156.0000 - 122ms/epoch - 2ms/step
Epoch 4/100
72/72 - 0s - loss: 2213143.2500 - mean_squared_error: 2213143.2500 - val_loss: 1531944.3750 - val_mean_squared_error: 1531944.3750 - 119ms/epoch - 2ms/step
Epoch 5/100
72/72 - 0s - loss: 2025706.6250 - mean_squared_error: 2025706.6250 - val_loss: 1476878.5000 - val_mean_squared_error: 1476878.5000 - 122ms/epoch - 2ms/step
Epoch 6/100
72/72 - 0s - loss: 1974734.6250 - mean_squared_error: 1974734.6250 - val_loss: 1511127.2500 - val_mean_squared_error: 1511127.2500 - 123ms/epoc

In [22]:
# evaluate the model - train
from sklearn.metrics import mean_squared_error

y_preds_train = model(X_train_seq)
print(f"MSE(train): {mean_squared_error(y_preds_train, y_train_seq)}")

MSE(train): 1248404.3137645612


In [23]:
# evaluate the model - test

y_preds_test = model(X_test_seq)
print(f"MSE(test): {mean_squared_error(y_preds_test, y_test_seq)}")

MSE(test): 1172888.0537396795
